In [5]:
import time
from dotenv import load_dotenv
load_dotenv()

import pandas as pd
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser

from langchain.output_parsers import OutputFixingParser

from langchain_google_genai import ChatGoogleGenerativeAI

In [2]:
proposal_df = pd.read_excel('data/pycon_2024_proposal.xlsx', engine="openpyxl")
reviews_df = pd.read_excel('data/pycon_2024_review.xlsx', engine="openpyxl")

/usr/local/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [3]:
proposal_df.id.is_unique

True

In [4]:
#4 RPM
proposal_info_columns = [ 'title'
  , 'abstract'
  , 'detailed_description'
  , 'outline'
  , 'objective']

result = []

with open("prompt/simple_prompt_david.txt", "r") as f:
    prompt = f.read()
prompt = PromptTemplate(input_variables=['PROPOSAL_INFO'],template=prompt)

#model = "gemini-2.0-pro-exp-02-05"
model = "gemini-2.0-flash-exp"
fixing_model = "gemini-2.0-flash-thinking-exp-01-21"

llm = ChatGoogleGenerativeAI(model=model, temperature=0)
fixing_llm = ChatGoogleGenerativeAI(model=fixing_model, temperature=0)

json_parser = JsonOutputParser()
fixing_parser = OutputFixingParser.from_llm(
    parser=json_parser,
    llm=fixing_llm
)

In [17]:
# chain = prompt | llm | StrOutputParser()
# proposal_info = proposal_df[proposal_info_columns].to_dict(orient='records')[0]
# executive_method = chain.invoke({"PROPOSAL_INFO": str(proposal_info)})


In [6]:
# 建立原始的 JsonOutputParser 和 fixing parser
processed_proposals = set(r.get('proposal_id') for r in result)

for proposal_id in proposal_df.id:
    start_time = time.time()
    if proposal_id in processed_proposals:
        print(f"Skipping already processed proposal: {proposal_id}")
        continue
        
    print(f"Processing proposal: {proposal_id}")
    proposal_info = proposal_df[proposal_df.id == proposal_id][proposal_info_columns].to_dict(orient='records')[0]
    
    while True:
        try:
            # Get raw output from LLM first
            chain = prompt | llm | StrOutputParser()
            raw_output = chain.invoke({"PROPOSAL_INFO": str(proposal_info)})
            
            try:
                # Try basic JSON parser first
                executive_method = json_parser.parse(raw_output)
            except Exception as e:
                print(f"Basic parser failed, attempting fix: {e}")
                # If basic parser fails, use fixing parser on the same output
                executive_method = fixing_parser.parse(raw_output)
            
            break
        except Exception as e:
            print(f"LLM or parsing failed for proposal {proposal_id}: {e}")
            time.sleep(10)
            continue
    
    executive_method['proposal_id'] = proposal_id
    result.append(executive_method)
    
    exec_time = time.time() - start_time
    print(f"Execution time for proposal {proposal_id}: {exec_time:.2f} seconds\n")

Processing proposal: 2178936233474917120
Execution time for proposal 2178936233474917120: 2.30 seconds

Processing proposal: 2178979854999880448
Execution time for proposal 2178979854999880448: 2.26 seconds

Processing proposal: 2178981707884004096
Execution time for proposal 2178981707884004096: 2.15 seconds

Processing proposal: 2179613228097602304
Execution time for proposal 2179613228097602304: 2.05 seconds

Processing proposal: 2180814504890204928
Execution time for proposal 2180814504890204928: 3.80 seconds

Processing proposal: 2181661100230050560
Execution time for proposal 2181661100230050560: 2.34 seconds

Processing proposal: 2181767915890541312
Execution time for proposal 2181767915890541312: 2.56 seconds

Processing proposal: 2183455404749488896
Execution time for proposal 2183455404749488896: 2.46 seconds

Processing proposal: 2184258951825064704
Execution time for proposal 2184258951825064704: 1.94 seconds

Processing proposal: 2184265081062163200
Execution time for prop

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..


LLM or parsing failed for proposal 2184271553351385856: 429 Resource has been exhausted (e.g. check quota).


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..


LLM or parsing failed for proposal 2184271553351385856: 429 Resource has been exhausted (e.g. check quota).


...

Execution time for proposal 2210750583986455296: 4.55 seconds

Processing proposal: 2210778103544808192


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..


LLM or parsing failed for proposal 2210778103544808192: 429 Resource has been exhausted (e.g. check quota).


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..


LLM or parsing failed for proposal 2210778103544808192: 429 Resource has been exhausted (e.g. check quota).
Execution time for proposal 2210778103544808192: 27.77 seconds

Processing proposal: 2211064418454733568
Execution time for proposal 2211064418454733568: 2.50 seconds



In [7]:
from collections import Counter

# Count occurrences of each proposal_id
Counter(r['vote'] for r in result)




Counter({'+1': 49, '+0': 18, '0': 9, '-0': 1})

In [12]:
[ x for x in result if x['proposal_id'] == 2202583607111844608 ]

[{'comment': "The proposal is well-structured and clearly outlines the benefits of using Python CDK for infrastructure deployment. The inclusion of a demo is a great way to engage the audience. Consider adding more specific examples of best practices and real-world use cases to enhance the session's practical value.",
  'vote': '+1',
  'proposal_id': 2202583607111844608}]

In [13]:
proposal_df[proposal_df.id == 2202583607111844608]

,id,title,category,duration,language,abstract,python_level,detailed_description,outline,supplementary,...,conference,last_updated_at,slido_embed_link,referring_policy,labels,first_time_speaker,live_stream_policy,hackmd_embed_link,living_in_taiwan,attend_in_person
35,2202583607111844608,Driving Efficiency through Automation: Leverag...,OTHER,PREF15,ENEN,Efficiently tackling some of the resource prov...,INTERMEDIATE,"During this session, I'll provide a comprehens...",1. Introduction: (2 minutes)\r\n . Briefly ...,"First and foremost, I want to express my grati...",...,pycontw-2024,2024-05-20 16:25:00.208,NaN,True,NaN,True,True,NaN,False,True
